# Natural Language Processing Introduction

- Demo for BADM 590
- Ye Joo Park and Unnati Narang

Natural language processing (NLP) aims to give computers the ability to understand, process, and even generate human language. This notebook introduces the common preprocessing steps and demonstrates how to use a widely used transformer model (`distilbert-base-uncased-finetuned-sst-2-english`) to perfrom sentiment analysis. 😀😦🙁

In [ ]:
# run this code cell before any other cell
# install pandas 2.2.0 if in colab environment
import sys

if 'google.colab' in sys.modules:
  !pip install pandas==2.2.0

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
pd.set_option('display.max_columns', 100)

## Load data

This exercise uses a small sampled dataset that contains reviews of property management companies in campustowns.

In [3]:
df_b = pd.read_csv(
    'https://github.com/bdi475/datasets/raw/main/campustowns-leasing-company-reviews/businesses-sample.csv'
)
df_r = pd.read_csv(
    'https://github.com/bdi475/datasets/raw/main/campustowns-leasing-company-reviews/reviews-sample.csv',
    parse_dates = ['review_datetime_utc', 'owner_answer_timestamp_datetime_utc']
)

In [4]:
display(df_b.head(2))
display(df_r.head(2))

,campus,place_id,name,site,category,borough,street,city,postal_code,state,latitude,longitude,verified
0,Indiana University Bloomington,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,State On Campus Bloomington,https://stateoncampus.com/bloomington/?utm_sou...,Apartment complex,NaN,2036 N Walnut St,Bloomington,47404,Indiana,39.184846,-86.532875,True
1,Indiana University Bloomington,ChIJPb8SbdpnbIgR82bkOLSKtZM,The Standard at Bloomington,https://www.thestandardbloomington.com/?utm_so...,Student housing center,NaN,250 E 14th St,Bloomington,47408,Indiana,39.175974,-86.531609,True


,place_id,review_id,author_id,author_title,review_text,review_rating,review_img_url,review_datetime_utc,owner_answer,owner_answer_timestamp_datetime_utc,review_likes
0,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,109839330111495228413,Aziz Bohra,"This place has my heart! Spacious rooms, quali...",5,https://lh5.googleusercontent.com/p/AF1QipM0Jm...,2023-04-01 03:19:15+00:00,Thanks for your feedback. We are grateful tha...,2022-11-01 18:54:45+00:00,0
1,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSUNaOC15NkhBEAE,102607480175477014087,Nessa Bacher,I’ll start with the positives of living at Sta...,4,NaN,2023-09-25 00:17:21+00:00,We are so pleased to hear that you enjoy livin...,2023-09-18 14:20:59+00:00,0


## 🪓 Tokenization using spaCy

In [5]:
import spacy

### Trained pipelines

Trained pipelines are models that enable spaCy to predict linguistic attributes in context

- Part-of-speech tags
- Syntactic dependencies
- Named entities

`'en_core_web_sm'` is a English pipeline optimized for CPU.

Components: 

- tok2vec
- taggerparser
- senter
- ner
- attribute_ruler
- lemmatizer

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
text = 'I love this apartment'
doc = nlp(text)

for token in doc:
    print('------------------')
    print(f'text: {token.text}')
    print(f'lemma: {token.lemma_}')
    print(f'pos: {token.pos_}') # pos_ stands for part-of-speech
    print(f'explain: {spacy.explain(token.pos_)}')
    print(f'is_stop: {token.is_stop}')

------------------
text: I
lemma: I
pos: PRON
explain: pronoun
is_stop: True
------------------
text: love
lemma: love
pos: VERB
explain: verb
is_stop: False
------------------
text: this
lemma: this
pos: DET
explain: determiner
is_stop: True
------------------
text: apartment
lemma: apartment
pos: NOUN
explain: noun
is_stop: False


We can visualize the parse tree using `displacy`.

In [8]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

### Tokenization and lemmatization

Tokenization takes a piece of text and breaks it down into meaningful units called "tokens." These tokens can be individual words, punctuation marks, numbers, or even phrases depending on the task and chosen method.

Lemmatization goes a step further, focusing on the "base form" or "dictionary form" of words. It groups together different grammatical variations of the same word (like "playing," "plays," "played") and reduces them to their core meaning ("play"). This helps capture the true meaning of the text regardless of how they are used.

In [9]:
cols = ["text", "lemma", "pos", "explain", "is_stop"]
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df_tokens = pd.DataFrame(rows, columns=cols)
df_tokens

,text,lemma,pos,explain,is_stop
0,I,I,PRON,pronoun,True
1,love,love,VERB,verb,False
2,this,this,DET,determiner,True
3,apartment,apartment,NOUN,noun,False


In [10]:
cols = ["review_id", "text", "lemma", "pos", "explain", "is_stop"]
rows = []

for index, row in df_r[df_r['review_text'].notna()].iterrows():
    doc = nlp(row['review_text'])
    for t in doc:
        new_row = [row['review_id'], t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
        rows.append(new_row)

df_tokens = pd.DataFrame(rows, columns=cols)
df_tokens

,review_id,text,lemma,pos,explain,is_stop
0,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,This,this,DET,determiner,True
1,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,place,place,NOUN,noun,False
2,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,has,have,VERB,verb,True
3,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,my,my,PRON,pronoun,True
4,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,heart,heart,NOUN,noun,False
...,...,...,...,...,...,...
18255,ChZDSUhNMG9nS0VJQ0FnSUNSdUpEV2FREAE,!,!,PUNCT,punctuation,False
18256,ChZDSUhNMG9nS0VJQ0FnSUNSeUp1dUVBEAE,Hooray,Hooray,PROPN,proper noun,False
18257,ChZDSUhNMG9nS0VJQ0FnSUNSeUp1dUVBEAE,!,!,PUNCT,punctuation,False
18258,ChZDSUhNMG9nS0VJQ0FnSUR4aDZQckl3EAE,wowie,wowie,PROPN,proper noun,False


### Remove stop words

Stop words, as you might guess from the name, are a set of commonly used words in a language that are often filtered out before processing text in Natural Language Processing (NLP) tasks. These words, like "the," "a," "is," "and," "on," etc., are considered to carry little independent meaning and contribute minimally to the overall understanding of the text.

We remove the stop words here for two reasons:

1. Reduce noise: By removing commonly used words, we focus on the content-rich keywords that convey the core meaning of the text.
2. Improve efficiency: Removing stop words reduces the overall size of the data, making NLP tasks faster and less computationally expensive.

In [11]:
# only filter non stop words
df_tokens_filtered = df_tokens[~df_tokens['is_stop']]

# remove words shorter than 4 characters long
df_tokens_filtered = df_tokens_filtered[df_tokens_filtered['lemma'].str.len() >= 4]

df_tokens_filtered

,review_id,text,lemma,pos,explain,is_stop
1,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,place,place,NOUN,noun,False
4,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,heart,heart,NOUN,noun,False
6,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,Spacious,spacious,ADJ,adjective,False
7,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,rooms,room,NOUN,noun,False
9,ChZDSUhNMG9nS0VJQ0FnSUMtbE1XaWR3EAE,quality,quality,NOUN,noun,False
...,...,...,...,...,...,...
18252,ChZDSUhNMG9nS0VJQ0FnSUNSdUpEV2FREAE,staff,staff,NOUN,noun,False
18254,ChZDSUhNMG9nS0VJQ0FnSUNSdUpEV2FREAE,awesome,awesome,ADJ,adjective,False
18256,ChZDSUhNMG9nS0VJQ0FnSUNSeUp1dUVBEAE,Hooray,Hooray,PROPN,proper noun,False
18258,ChZDSUhNMG9nS0VJQ0FnSUR4aDZQckl3EAE,wowie,wowie,PROPN,proper noun,False


Display value counts.

In [12]:
df_tokens_filtered['lemma'].value_counts()

lemma
apartment    125
live         119
place         99
staff         71
great         62
            ... 
discard        1
needle         1
massive        1
forever        1
zowa           1
Name: count, Length: 1457, dtype: int64

## 🧪 Sentiment analysis using distilbert

From [Hugging Face's Documentation](https://huggingface.co/docs/transformers/main/en/index):

> Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as:

> 📝 **Natural Language Processing**: text classification, named entity recognition, question answering, language modeling, summarization, translation, multiple choice, and text generation.

In [13]:
! pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [14]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

2024-02-21 22:05:41.401777: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 22:05:42.238292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 22:05:42.238450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 22:05:42.401335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 22:05:42.776190: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 22:05:42.781856: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Run sentiment classifier

In [15]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [16]:
classifier("These thieves tried to steal my security deposit.")

[{'label': 'NEGATIVE', 'score': 0.996752142906189}]

### Sample 30 rows for in-class exercise

Although the `distilbert-base-uncased-finetuned-sst-2-english` model is pre-trained and distilled (40% smaller than the original BERT model), it will still be slow to be used for the entire dataset.

For in-class exercise, only sample 30 rows where review_rating is less than or equal to 4 out of 5 stars.

In [17]:
df_sample = df_r[df_r['review_rating'] <= 4].sample(30)

Create two new columns to store sentiment labels and scores.

In [18]:
df_sample['sentiment'] = np.nan
df_sample['score'] = np.nan

Run sentiment analysis.

In [19]:
num_rows = df_sample.shape[0]

for i in range(num_rows):
  # grab review text to a variable
  review_text = df_sample['review_text'].iloc[i]

  # calculate sentiments for non-missing review texts
  # the model supports up to 512 tokens
  # truncate longer texts
  if pd.notna(review_text):
    result = classifier(
        review_text,
        truncation=True,
        padding=True,
        max_length=512
    )
    
    df_sample.iloc[i, df_sample.columns.get_loc('sentiment')] = result[0]['label']
    df_sample.iloc[i, df_sample.columns.get_loc('score')] = result[0]['score']

  # display progress
  progress_percentage = round((i + 1) / num_rows * 100, 2)
  print(f'{i + 1}/{num_rows} ({progress_percentage}%)', end=' ')

  if (i + 1) % 10 == 0:
    print('')

print('====================')
print('Complete')

/tmp/ipykernel_2491771/4191358781.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'POSITIVE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sample.iloc[i, df_sample.columns.get_loc('sentiment')] = result[0]['label']


1/30 (3.33%) 2/30 (6.67%) 3/30 (10.0%) 4/30 (13.33%) 5/30 (16.67%) 6/30 (20.0%) 7/30 (23.33%) 8/30 (26.67%) 9/30 (30.0%) 10/30 (33.33%) 
11/30 (36.67%) 12/30 (40.0%) 13/30 (43.33%) 14/30 (46.67%) 15/30 (50.0%) 16/30 (53.33%) 17/30 (56.67%) 18/30 (60.0%) 19/30 (63.33%) 20/30 (66.67%) 
21/30 (70.0%) 22/30 (73.33%) 23/30 (76.67%) 24/30 (80.0%) 25/30 (83.33%) 26/30 (86.67%) 27/30 (90.0%) 28/30 (93.33%) 29/30 (96.67%) 30/30 (100.0%) 
Complete


In [20]:
df_sample

,place_id,review_id,author_id,author_title,review_text,review_rating,review_img_url,review_datetime_utc,owner_answer,owner_answer_timestamp_datetime_utc,review_likes,sentiment,score
110,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSUNpdFB1Skp3EAE,111669024045891112596,Isaiah Reed,Office staff was great every time I had a ques...,4,NaN,2021-07-28 16:38:41+00:00,Thank you for the kind review. If there is any...,2021-10-15 14:00:25+00:00,0,POSITIVE,0.999755
152,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChdDSUhNMG9nS0VJQ0FnSUQ2NWNYeTB3RRAB,118173170171218260785,Adrian Guerrero,NaN,1,NaN,2021-10-06 04:22:39+00:00,"Adrian, we really enjoyed having you stay with...",2021-10-15 14:00:32+00:00,0,NaN,NaN
83,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSUMwN0lDckRnEAE,117288863178358765301,WMW W,"Apartments are very nice, especially the 3 be...",1,NaN,2019-09-17 16:54:26+00:00,"Good evening, like we mentioned in our two pho...",2019-09-18 02:43:27+00:00,7,NEGATIVE,0.974977
187,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSUNGLTZQeUdREAE,103150992701781783867,Jordan Pettay,NaN,4,NaN,2023-11-03 09:13:25+00:00,Thanks for your feedback. We are grateful that...,2023-11-08 14:54:16+00:00,0,NaN,NaN
9,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSURlZ2U2WEVREAE,111470724394028227478,Jackie L.,NEVER live here. Easily the worst experience o...,1,NaN,2022-10-11 18:44:53+00:00,"Jackie, thank you for your comment. We strive ...",2022-10-12 13:57:31+00:00,7,NEGATIVE,0.999705
15,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChdDSUhNMG9nS0VJQ0FnSUR1eTRxUDhBRRAB,114139735823779959640,Noelle Humphrey,If I could give them 0 stars I would. I sublea...,1,NaN,2022-08-29 15:51:49+00:00,"Noelle, thank you for your feedback. We hope t...",2022-08-30 12:32:33+00:00,6,NEGATIVE,0.999707
125,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSURVcTVuOVZBEAE,102675320260316251241,V,Very prompt response to an email needing maint...,4,NaN,2019-09-04 16:56:28+00:00,NaN,NaT,0,POSITIVE,0.998680
157,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChdDSUhNMG9nS0VJQ0FnSUNlN3EyR3hBRRAB,108951189460210295502,Kendall Brooke,NaN,1,NaN,2022-09-14 16:06:14+00:00,"Hi there, Kendall. We cannot find you in our s...",2022-09-16 14:14:28+00:00,0,NaN,NaN
93,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChdDSUhNMG9nS0VJQ0FnSUNha05DRTN3RRAB,105330555025255804499,Nava Rapp,The pipe burst this apartment upstairs office ...,1,NaN,2021-07-21 09:47:41+00:00,NaN,NaT,4,NEGATIVE,0.999753
171,ChIJY1yB5NJmbIgRZn7E2oF5gVQ,ChZDSUhNMG9nS0VJQ0FnSURwbFpiNGR3EAE,111446913814811258301,I like Pie,NaN,1,NaN,2023-08-26 01:58:04+00:00,"Hi there, We cannot find you in our system. Wo...",2023-08-28 20:17:24+00:00,0,NaN,NaN
